In [24]:
# ================= 0.0 Pacotes e Bibliotecas =================

# Manipulação de Dados
import pandas as pd
import numpy as np

# Salvar modelos
import pickle as pkl


# ================== 1.0 Carregamento dos Dados ==================
df1 = pd.read_parquet('../BaseDadosProducao/BaseDadosProducao.parquet', engine='fastparquet')

# ================== 2.0 Tratamentos dos Dados ==================
# Cópia dos Dados
df3 = df1.copy()

# Eliminando as colunas desnecessárias
df3 = df3.drop(columns=['day', 'month'], axis=1).copy()

# Tratamento da Variável Job
df3['job'] = np.where(df3['job'] == 'unknown', 'unknown_job', df3['job'])

# Tratamento da Variável Education
df3['education'] = np.where(df3['job'] == 'unknown', 'unknown_education', df3['education'])

# Tratamento da Variável Contact
df3['contact'] = np.where(df3['contact'] == 'unknown', 'unknown_contact', df3['contact'])

# Tratamento da Variável poutcome
df3['poutcome'] = np.where(df3['poutcome'] == 'unknown', 'unknown_outcome', df3['poutcome'])

# Obtendo os dados dos clientes cuja duração de ligações são mais que 30 segundos.
df3 = df3[df3['duration'] >= 30].copy()

# ================== 4.0 Engenharia de Atributos ==================
df4 = df3.copy()

# Engenharia da variável pdays
df4['pdays'] = np.where(df4['pdays'] == -1, 'First Contact',
                np.where(np.trunc(df4['pdays']/30) == 1, 'one month without contact',
                np.where(np.trunc(df4['pdays']/30) == 2, 'two months without contact',
                np.where(np.trunc(df4['pdays']/30) == 3, 'three months without contact',
                np.where((3 < np.trunc(df4['pdays']/30)) & (np.trunc(df4['pdays']/30) <= 6), '+3 months without contact',
                np.where((6 < np.trunc(df4['pdays']/30)) & (np.trunc(df4['pdays']/30) <= 12), '+6 months without contact',
                         '+1 year without contact'))))))

# ================== 5.0 Carregamento do Modelo ==================

modelo = pkl.load(open('../ModeloProducao/ModeloAceitacaoContratoV1.pkl', 'rb'))

# ================== 6.0 Predicões ==================

predicoes = modelo.predict_proba(df4)

# ================== 7.0 Tratativas e Ajustes Finais ==================
df4['Probabilidades'] = predicoes[:, 1]

base_acoes = df4.sort_values(by=['Probabilidades'], ascending=False)

# ================== 8.0 Salvando os Dados ==================
base_acoes.to_parquet('../BaseDadosProducao/BaseAcoes.parquet', index=False, engine='fastparquet')